Train The Model

In [ ]:
length_of_training_dataset = 3000
length_of_validation_dataset = 3033
steps_per_epoch = length_of_training_dataset // BATCH_SIZE
validation_steps = length_of_training_dataset // BATCH_SIZE

if length_of_validation_dataset % BATCH_SIZE >0:
    validation_steps += 1

In [ ]:
history = model.fit(training_dataset, 
                    epochs = 50,
                    steps_per_epoch= steps_per_epoch,
                    validation_data=validation_dataset,
                    validation_steps = validation_steps)

Evaluate Using IoU Method

In [ ]:
def intersection_over_union(pred_box, true_box):
    xmin_pred, ymin_pred, xmax_pred, ymax_pred = np.split(pred_box, 4, axis = 1)
    xmin_true, ymin_true, xmax_true, ymax_true = np.split(true_box, 4, axis=1)
    
    
    xmin_overlap = np.maximum(xmin_pred, xmin_true)
    xmax_overlap = np.minimum(xmax_pred, xmax_true)
    ymin_overlap = np.maximum(ymin_pred, ymin_true)
    ymax_overlap = np.minimum(ymax_pred, ymax_true)
    
    pred_box_area = (xmax_pred - xmin_pred) * (ymax_pred - ymin_pred)
    true_box_area = (xmax_true - xmin_true) * (ymax_true - ymin_true)
    
    overlap_area = np.maximum((xmax_overlap - xmin_overlap),0) * np.maximum((ymax_overlap - ymin_overlap), 0)
    union_area = (pred_box_area + true_box_area) - overlap_area
    
    smoothing_factor = 1e-10
    
    iou = (overlap_area + smoothing_factor) / (union_area + smoothing_factor)
    
    return iou

    

In [ ]:
original_images, normalized_images, normalized_bboxes = dataset_to_numpy_with_original_bboxes_util(visualization_validation_dataset, N=500)
predicted_bboxes = model.predict(normalized_images, batch_size=32)
iou = intersection_over_union(predicted_bboxes, normalized_bboxes)
iou_threshold = 0.5
print("Number of predictions where iou > threshold(%s): %s" % (iou_threshold, (iou >= iou_threshold).sum()))
print("Number of predictions where iou < threshold(%s): %s" % (iou_threshold, (iou < iou_threshold).sum()))


Visualize Predictions

In [ ]:
n = 15
indexes = np.random.choice(len(predicted_bboxes), size=n)
iou_to_draw = iou[indexes]
norm_to_draw = original_images[indexes]
display_digits_with_boxes(original_images[indexes], predicted_bboxes[indexes],normalized_bboxes[indexes], iou[indexes], "True and Predicted values", bboxes_normalized=True)